In [1]:
import sqlite3
import pandas as pd
from datetime import datetime
import os

# Nom de la base
DB_NAME = f"{os.path.abspath('.')}/KLBrvm_BataBase.db"

# Créer la base et les tables si elles n'existent pas
def init_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Création de la table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_prices (
        Date TEXT NOT NULL,
        Ticker TEXT NOT NULL,
        Price REAL,
        PRIMARY KEY (Date, Ticker)
    )
    """)
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS capitalisations (
            Ticker TEXT,
            Stock_Description TEXT,
            Nombre_de_titres REAL,
            Cours_du_jour REAL,
            Capitalisation_flottante REAL,
            Capitalisation_globale REAL,
            Capitalisation_globale_pct REAL,
            Date TEXT,
            PRIMARY KEY (Ticker, Date)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS per_data (
            Ticker TEXT,
            Stock_Description TEXT,
            Nombre_titres_echanges REAL,
            Valeur_echangee REAL,
            PER REAL,
            Valeur_globale_echangee_pct REAL,
            Date TEXT,
            PRIMARY KEY (Ticker, Date)
        )
    ''')
    
    conn.commit()
    conn.close()

# Insère un DataFrame dans une table SQLite
def insert_dataframe(df: pd.DataFrame, table_name: str):
    conn = sqlite3.connect(DB_NAME)
    df.reset_index(inplace=True)
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()

# Exemple d’usage avec tes fonctions existantes
def store_daily_data(get_data_today):
    df_price, df_cap, df_per = get_data_today()
    
    insert_dataframe(df_price, "stock_prices")
    insert_dataframe(df_cap, "capitalisations")
    insert_dataframe(df_per, "per_data")

# Initialisation une seule fois

# Ensuite on lance ceci chaque jour
#store_daily_data()


In [2]:
init_db()

In [3]:

def load_available_data(self):
    df_p = pd.read_excel("../Stocks_Prices/Global_Historic/Cours_titres.xlsx")
    df_sorted = df_p.sort_index(ascending=False)
    df_sorted.columns = ["Date"] + list(df_sorted.columns[1:])
    df_melt = df_sorted.melt(id_vars='Date', var_name='Ticker', value_name='Price')
    #df_sorted = df_sorted.drop("Date/Société", axis=1)
    # Insertion
    self._connect()
    df_melt.to_sql("stock_prices", conn, if_exists="append", index=False)
    self.close()


In [6]:
df_melt.head()

,Date,Ticker,Price
0,2025-03-18,ABJC,1840.0
1,2025-03-17,ABJC,1810.0
2,2025-03-14,ABJC,1800.0
3,2025-03-13,ABJC,1800.0
4,2025-03-12,ABJC,1850.0


In [7]:
# Connexion à la base
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()


# Transformation du DataFrame
#df_prices = df_prices.reset_index()  # assure que 'date' est une colonne
df_melt = df_sorted.melt(id_vars='Date', var_name='Ticker', value_name='Price')

# Insertion
df_melt.to_sql("stock_prices", conn, if_exists="append", index=False)

conn.commit()
conn.close()


In [13]:
df_melt

,Date,Ticker,Price
0,2025-03-18,ABJC,1840.0
1,2025-03-17,ABJC,1810.0
2,2025-03-14,ABJC,1800.0
3,2025-03-13,ABJC,1800.0
4,2025-03-12,ABJC,1850.0
...,...,...,...
145131,2013-01-08,UNXC,120.0
145132,2013-01-07,UNXC,120.0
145133,2013-01-04,UNXC,120.0
145134,2013-01-03,UNXC,120.0


In [16]:
DB_NAME

'KLBrvm_BataBase.db'

In [8]:
def get_prices_from_db(ticker, start_date=None, end_date=None, db_path=DB_NAME):
    
    # Récupère les prix pour un ticker dans une plage de dates
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    query = f"SELECT * FROM stock_prices WHERE Ticker = ?"

    params = [ticker]
    
    if start_date:
        query += " AND Date >= ?"
        params.append(start_date)
    if end_date:
        query += " AND Date <= ?"
        params.append(end_date)
    
    cursor.execute(query, tuple(params))
    result = cursor.fetchall()
    #print(result)
    conn.close()

    # Conversion en DataFrame pour une meilleure manipulation
    columns = ['Date', 'Ticker', 'Price']
    conn.close()
    return pd.DataFrame(result, columns=columns)

# Initialisation une seule fois

# Ensuite on lance ceci chaque jour
#store_daily_data()

def ExecuteQuery(query, db_path=DB_NAME):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return result

In [18]:
get_prices_from_db("ORGT", "2025-02-17", "2025-04-19")

,Date,Ticker,Price
0,2025-02-17 00:00:00,ORGT,1675.0
1,2025-02-18 00:00:00,ORGT,1665.0
2,2025-02-19 00:00:00,ORGT,1665.0
3,2025-02-20 00:00:00,ORGT,1595.0
4,2025-02-21 00:00:00,ORGT,1595.0
5,2025-02-24 00:00:00,ORGT,1595.0
6,2025-02-25 00:00:00,ORGT,1595.0
7,2025-02-26 00:00:00,ORGT,1595.0
8,2025-02-27 00:00:00,ORGT,1595.0
9,2025-02-28 00:00:00,ORGT,1550.0


In [10]:
ExecuteQuery("SELECT DISTINCT ticker FROM stock_prices;")

[('ABJC',),
 ('BICC',),
 ('BNBC',),
 ('BOAB',),
 ('BOABF',),
 ('BOAC',),
 ('BOAM',),
 ('BOAN',),
 ('BOAS',),
 ('CABC',),
 ('CBIBF',),
 ('CFAC',),
 ('CIEC',),
 ('ECOC',),
 ('ETIT',),
 ('FTSC',),
 ('LNBB',),
 ('NEIC',),
 ('NSBC',),
 ('NTLC',),
 ('ONTBF',),
 ('ORAC',),
 ('ORGT',),
 ('PALC',),
 ('PRSC',),
 ('SAFC',),
 ('SCRC',),
 ('SDCC',),
 ('SDSC',),
 ('SEMC',),
 ('SGBC',),
 ('SHEC',),
 ('SIBC',),
 ('SICC',),
 ('SIVC',),
 ('SLBC',),
 ('SMBC',),
 ('SNTS',),
 ('SOGC',),
 ('SPHC',),
 ('STAC',),
 ('STBC',),
 ('SVOC',),
 ('TTLC',),
 ('TTLS',),
 ('UNLC',),
 ('UNXC',)]

In [11]:
conn = sqlite3.connect(DB_NAME)
df = pd.read_sql("SELECT * FROM stock_prices LIMIT 10", conn)
print(df)

                  Date Ticker   Price
0  2025-03-18 00:00:00   ABJC  1840.0
1  2025-03-17 00:00:00   ABJC  1810.0
2  2025-03-14 00:00:00   ABJC  1800.0
3  2025-03-13 00:00:00   ABJC  1800.0
4  2025-03-12 00:00:00   ABJC  1850.0
5  2025-03-11 00:00:00   ABJC  1800.0
6  2025-03-10 00:00:00   ABJC  1795.0
7  2025-03-07 00:00:00   ABJC  1870.0
8  2025-03-06 00:00:00   ABJC  1870.0
9  2025-03-05 00:00:00   ABJC  1825.0


In [12]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('stock_prices',), ('capitalisations',), ('per_data',)]


In [28]:
df = pd.read_sql("SELECT * FROM stock_prices", conn)
print(df.head())
print(df.columns)

Empty DataFrame
Columns: [Date, Ticker, Price]
Index: []
Index(['Date', 'Ticker', 'Price'], dtype='object')


In [3]:
import os
os.listdir()

['.ipynb_checkpoints', 'DBUtilities.py', 'init_db.ipynb', '__pycache__']

In [15]:
import os
import sqlite3
import pandas as pd
from datetime import datetime

class BRVMDatabase:
    def __init__(self, db_name="KLBrvm_BataBase.db", root_path="C:\\Users\\charl\\Documents\\KLDocs\\Investment\\DataBase"):
        self.root = root_path
        self.db_name = f"{self.path}\\{db_name}"
        self._init_db()
        self._connect()

    def _connect(self):
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        #return self.conn, self.cursor

    def _init_db(self, force=False):

        if os.path.exists("KLBrvm_BataBase.db") and not force:
            return 0
        
        self._connect()

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS stock_prices (
            Date TEXT NOT NULL,
            Ticker TEXT NOT NULL,
            Price REAL,
            PRIMARY KEY (Date, Ticker)
        )
        """)

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS capitalisations (
                Ticker TEXT,
                Stock_Description TEXT,
                Nombre_de_titres REAL,
                Cours_du_jour REAL,
                Capitalisation_flottante REAL,
                Capitalisation_globale REAL,
                Capitalisation_globale_pct REAL,
                Date TEXT,
                PRIMARY KEY (Ticker, Date)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS per_data (
                Ticker TEXT,
                Stock_Description TEXT,
                Nombre_titres_echanges REAL,
                Valeur_echangee REAL,
                PER REAL,
                Valeur_globale_echangee_pct REAL,
                Date TEXT,
                PRIMARY KEY (Ticker, Date)
            )
        ''')

        self.conn.commit()
        self.close()
        

    def validate_dataframe(self, df: pd.DataFrame, table: str):
        required_fields = {
            'stock_prices': {'Date', 'Ticker', 'Price'},
            'capitalisations': {'Date', 'Ticker', 'Stock_Description'},
            'per_data': {'Date', 'Ticker', 'Stock_Description'}
        }
        if table not in required_fields:
            raise ValueError("Table inconnue")
        df_cols = set(df.columns)
        missing = required_fields[table] - df_cols
        if missing:
            raise ValueError(f"Colonnes manquantes pour {table}: {missing}")

        if 'Date' in df.columns:
            try:
                pd.to_datetime(df['Date'])
            except Exception as e:
                raise ValueError("Date non convertible: " + str(e))

    def insert_dataframe(self, df: pd.DataFrame, table: str):
        self.validate_dataframe(df, table)
        df = df.copy()
        df.reset_index(inplace=True, drop=False) if df.index.name in ["Date", "Ticker"] else None
        self._connect()
        try:
            df.to_sql(table, self.conn, if_exists='append', index=False)
        except Exception as e:
            raise RuntimeError(f"Erreur d'insertion: {e}")
        finally:
            self.close()
        self.close()
        
    def insert_dataframe2(self, df: pd.DataFrame, table_name: str):
    if df.empty:
        print("[WARNING] DataFrame is empty.")
        return

    required_cols = set(self._get_table_columns(table_name))
    df = df.reset_index(drop=True)  # ⚠️ Important pour éviter l'insertion d'index
    incoming_cols = set(df.columns)

    if not required_cols.issubset(incoming_cols):
        raise ValueError(f"[ERROR] Incoming columns do not match table '{table_name}' schema. Required: {required_cols}, Got: {incoming_cols}")

    df = df[list(required_cols)]  # Assure l'ordre et filtre si besoin
    df.to_sql(table_name, self.conn, if_exists='append', index=False)


    def get_prices(self, tickers=None, start_date=None, end_date=None):
        query = "SELECT * FROM stock_prices WHERE 1=1"
        params = []
        if tickers:
            query += " AND Ticker IN ({})".format(",".join(["?"] * len(tickers)))
            params.extend(tickers)
        if start_date:
            query += " AND Date >= ?"
            params.append(start_date)
        if end_date:
            query += " AND Date <= ?"
            params.append(end_date)

        self._connect()
        df = pd.read_sql_query(query, self.conn, params=params, parse_dates=['Date'])
        self.close()
        return df

    def clear_table(self, table):
        self._connect()
        self.cursor.execute(f"DELETE FROM {table}")
        self.conn.commit()
        self.close()

    def status(self, tables = ['stock_prices', 'capitalisations', 'per_data']):
        self._connect()
        print("Table status (rows):")
        for t in tables:
            self.cursor.execute(f"SELECT COUNT(*) FROM {t}")
            print(f"{t} : {self.cursor.fetchone()[0]} rows")
        self.close()

    def delete_last_insert(self, table):
        self._connect()
        cursor.execute(f"SELECT MAX(Date) FROM {table}")
        last_date = cursor.fetchone()[0]
        if last_date:
            cursor.execute(f"DELETE FROM {table} WHERE Date = ?", (last_date,))
            print(f"Supprimée: {table} pour la date {last_date}")
        conn.commit()
        conn.close()
        
    def execute_query(self, query: str, params: tuple = ()):
        conn, cursor = self._connect()
        self.cursor.execute(query, params)
        self.conn.commit()

    def get_value(self, table: str, columns='*', condition: str = '', order_by: str = '', limit: int = None):
        conn, cursor = self._connect()
        query = f"SELECT {columns} FROM {table}"
        if condition:
            query += f" WHERE {condition}"
        if order_by:
            query += f" ORDER BY {order_by}"
        if limit:
            query += f" LIMIT {limit}"
        return pd.read_sql_query(query, self.conn)

    def get_all_tables(self):
        query = "SELECT name FROM sqlite_master WHERE type='table'"
        self._connect()
        res = [row[0] for row in self.cursor.execute(query)]
        return res

    def set_value(self, table: str, set_clause: str, condition: str):
        query = f"UPDATE {table} SET {set_clause} WHERE {condition}"
        self.execute_query(query)

    def delete_value(self, table: str, condition: str):
        query = f"DELETE FROM {table} WHERE {condition}"
        self.execute_query(query)

    def delete_and_create(self, table: str = None):
        tables = self.get_all_tables() if table is None else [table]
        self._connect()
        for t in tables:
            self.cursor.execute(f"DROP TABLE IF EXISTS {t}")
        self.conn.commit()
        self._init_db()
        self.close()

    def _get_table_columns(self, table: str):
        self.cursor.execute(f"PRAGMA table_info({table})")
        return [row[1] for row in self.cursor.fetchall()]

    def status(self):
        tables = self.get_all_tables()
        self._connect()
        print("[DATABASE STATUS]")
        for t in tables:
            self.cursor.execute(f"SELECT COUNT(*) FROM {t}")
            count = self.cursor.fetchone()[0]
            print(f"Table '{t}': {count} rows")

    def close(self):
        self.conn.close()
        
    def load_available_data(self):
    df_p = pd.read_excel(f"{self.path}/../Stocks_Prices/Global_Historic/Cours_titres.xlsx")
    df_sorted = df_p.sort_index(ascending=False)
    df_sorted.columns = ["Date"] + list(df_sorted.columns[1:])
    df_melt = df_sorted.melt(id_vars='Date', var_name='Ticker', value_name='Price')
    #df_sorted = df_sorted.drop("Date/Société", axis=1)
    # Insertion
    self._connect()
    df_melt.to_sql("stock_prices", conn, if_exists="append", index=False)
    self.close()


